![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

**Read in bank_marketing.csv as a pandas DataFrame.**

In [260]:
import pandas as pd
import numpy as np

# Start coding here...
df = pd.read_csv('bank_marketing.csv')
df.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,13,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,19,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,23,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,27,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,3,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


**Split the data into three DataFrames using information provided about the desired tables as your guide: one with information about the client, another containing campaign data, and a third to store information about economics at the time of the campaign.**

In [261]:
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]
client.head()

,client_id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [262]:
campaign = df[['campaign', 'client_id', 'contact', 'duration', 'pdays', 'previous', 'poutcome', 'month', 'day', 'y']]
campaign.head()

,campaign,client_id,contact,duration,pdays,previous,poutcome,month,day,y
0,1,0,telephone,261,999,0,nonexistent,may,13,no
1,1,1,telephone,149,999,0,nonexistent,may,19,no
2,1,2,telephone,226,999,0,nonexistent,may,23,no
3,1,3,telephone,151,999,0,nonexistent,may,27,no
4,1,4,telephone,307,999,0,nonexistent,may,3,no


In [263]:
economics = df[['emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed']]
economics.head()

,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
0,1.1,93.994,-36.4,4.857,5191.0
1,1.1,93.994,-36.4,4.857,5191.0
2,1.1,93.994,-36.4,4.857,5191.0
3,1.1,93.994,-36.4,4.857,5191.0
4,1.1,93.994,-36.4,4.857,5191.0


**Rename the column "client_id" to "id" in client (leave as-is in the other subsets); "duration" to "contact_duration", "previous" to "previous_campaign_contacts", "y" to "campaign_outcome", "poutcome" to "previous_outcome", and "campaign" to "number_contacts" in campaign; and "euribor3m" to "euribor_three_months" and "nr_employed" to "number_employed" in economics.**

In [264]:
client.rename(columns={'client_id':'id'}, inplace=True)
print(client.columns.tolist()) 


['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']


In [265]:
campaign.rename(columns={'duration':'contact_duration', 'previous':'previous_campaign_contacts', 'y':'campaign_outcome', 'poutcome':'previous_outcome', 'campaign':'number_contacts'}, inplace=True)
print(campaign.columns.tolist()) 


['number_contacts', 'client_id', 'contact', 'contact_duration', 'pdays', 'previous_campaign_contacts', 'previous_outcome', 'month', 'day', 'campaign_outcome']


In [266]:
economics.rename(columns={'euribor3m':'euribor_three_months', 'nr_employed': 'number_employed'}, inplace=True)
print(economics.columns.tolist())

['emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor_three_months', 'number_employed']


**Clean the "education" column, changing "." to "_" and "unknown" to NumPy's null values.**

In [267]:
client['education'] = client['education'].str.replace('.', '_') 
client['education'] = client['education'].replace({'unknown':np.nan})
client['education'].head(10)

0               basic_4y
1            high_school
2            high_school
3               basic_6y
4            high_school
5               basic_9y
6    professional_course
7                    NaN
8    professional_course
9            high_school
Name: education, dtype: object

**Remove periods from the "job" column.**

In [268]:
client['job'] = client['job'].str.replace('.','')
client['job'].head(10)

0      housemaid
1       services
2       services
3          admin
4       services
5       services
6          admin
7    blue-collar
8     technician
9       services
Name: job, dtype: object

**Convert "success" and "failure" in the "previous_outcome" and "campaign_outcome" columns to binary (1 or 0), along with the changing "nonexistent" to NumPy's null values in "previous_outcome".**

In [269]:
campaign['previous_outcome'] = campaign['previous_outcome'].replace({'success': 1, 'failure': 0})
campaign['campaign_outcome'] = campaign['campaign_outcome'].replace({'success': 1, 'failure': 0})
campaign['previous_outcome'] = campaign['previous_outcome'].replace({'nonexistent': np.nan})
campaign.head(10)

,number_contacts,client_id,contact,contact_duration,pdays,previous_campaign_contacts,previous_outcome,month,day,campaign_outcome
0,1,0,telephone,261,999,0,NaN,may,13,no
1,1,1,telephone,149,999,0,NaN,may,19,no
2,1,2,telephone,226,999,0,NaN,may,23,no
3,1,3,telephone,151,999,0,NaN,may,27,no
4,1,4,telephone,307,999,0,NaN,may,3,no
5,1,5,telephone,198,999,0,NaN,may,5,no
6,1,6,telephone,139,999,0,NaN,may,3,no
7,1,7,telephone,217,999,0,NaN,may,12,no
8,1,8,telephone,380,999,0,NaN,may,21,no
9,1,9,telephone,50,999,0,NaN,may,5,no


**Add a column called campaign_id in campaign, where all rows have a value of 1**

In [270]:
campaign['campaign_id']=1
campaign.head(10)

,number_contacts,client_id,contact,contact_duration,pdays,previous_campaign_contacts,previous_outcome,month,day,campaign_outcome,campaign_id
0,1,0,telephone,261,999,0,NaN,may,13,no,1
1,1,1,telephone,149,999,0,NaN,may,19,no,1
2,1,2,telephone,226,999,0,NaN,may,23,no,1
3,1,3,telephone,151,999,0,NaN,may,27,no,1
4,1,4,telephone,307,999,0,NaN,may,3,no,1
5,1,5,telephone,198,999,0,NaN,may,5,no,1
6,1,6,telephone,139,999,0,NaN,may,3,no,1
7,1,7,telephone,217,999,0,NaN,may,12,no,1
8,1,8,telephone,380,999,0,NaN,may,21,no,1
9,1,9,telephone,50,999,0,NaN,may,5,no,1


In [271]:
campaign['month'].head(20)

0     may
1     may
2     may
3     may
4     may
5     may
6     may
7     may
8     may
9     may
10    may
11    may
12    may
13    may
14    may
15    may
16    may
17    may
18    may
19    may
Name: month, dtype: object

**Create a datetime column called last_contact_date, in the format of "year-month-day", where the year is 2022, and the month and day values are taken from the "month" and "day" columns.**


In [272]:
# Create a new column containing the year
campaign['year'] = 2022

# Convert month names to their corresponding numeric values
month_dict = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
campaign['month'] = campaign['month'].replace(month_dict)

# Create a new datetime column from the 'year', 'month', and 'day' columns
campaign['last_contact_date'] = pd.to_datetime(campaign[['year', 'month', 'day']])
campaign['last_contact_date'] = campaign['last_contact_date'].dt.strftime('%Y-%m-%d')

print("\nModified DataFrame")
campaign.head()


Modified DataFrame


,number_contacts,client_id,contact,contact_duration,pdays,previous_campaign_contacts,previous_outcome,month,day,campaign_outcome,campaign_id,year,last_contact_date
0,1,0,telephone,261,999,0,NaN,5,13,no,1,2022,2022-05-13
1,1,1,telephone,149,999,0,NaN,5,19,no,1,2022,2022-05-19
2,1,2,telephone,226,999,0,NaN,5,23,no,1,2022,2022-05-23
3,1,3,telephone,151,999,0,NaN,5,27,no,1,2022,2022-05-27
4,1,4,telephone,307,999,0,NaN,5,3,no,1,2022,2022-05-03


**Remove any redundant data that might have been used to create new columns, ensuring the columns in each subset of the data match the table displayed in the notebook.**

In [273]:
campaign = campaign.drop(['month', 'day', 'year'], axis=1)
campaign.head()

,number_contacts,client_id,contact,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,1,0,telephone,261,999,0,NaN,no,1,2022-05-13
1,1,1,telephone,149,999,0,NaN,no,1,2022-05-19
2,1,2,telephone,226,999,0,NaN,no,1,2022-05-23
3,1,3,telephone,151,999,0,NaN,no,1,2022-05-27
4,1,4,telephone,307,999,0,NaN,no,1,2022-05-03


**Save the three DataFrames to csv files without an index as client.csv, campaign.csv, and economics.csv respectively.**

In [274]:
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)

**Create a variable called client_table, containing SQL code as a string to create a table called client using values from client.csv**

In [275]:
client_table = """
CREATE TABLE client (
    id	serial PRIMARY KEY,
    age	integer,	
    job	text,
    marital	text,	
    education text,	
    credit_default	boolean
    housing	boolean	
    loan boolean	
);

\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

**Create a variable called campaign_table, containing SQL code as a string to create a table called campaign using values from campaign.csv**

In [276]:
campaign_table = """
CREATE TABLE campaign (
    campaign_id	serial PRIMARY KEY
    client_id serial REFERENCES client (id)
    number_contacts	integer,
    contact	integer,	
    contact_duration integer,
    pdays integer,	
    previous_campaign_contacts integer,	
    previous_outcome boolean
    campaign_outcome boolean	
        last_contact_date date
);

\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

**Create a variable called economics_table, containing SQL code as a string to create a table called economics using values from economics.csv.**

In [277]:
economics_table = """
CREATE TABLE economics (
    client_id serial REFERENCES client (id)
    emp_var_rate	float	
    cons_price_idx	float	
    euribor_three_months	float	
    number_employed	float	
);

\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""

**In client, campaign, and economic, ensure the final line copies the data from their respective csv files using the following code snippet:
\copy table_name from 'file_name.csv' DELIMITER ',' CSV HEADER**